# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
from tqdm import tqdm

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')
nltk.download('punkt')

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'https?://\S+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\S+', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@\S+', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = nltk.tokenize.word_tokenize(text)

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(token) for token in tokens if len(token) > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement
import pickle
import os.path

tokenized_twits = []
filename = 'tokenList.txt'
if os.path.exists(filename):
    print('Load preprocessed twits')
    with open(filename, 'rb') as fp:
        tokenized_twits = pickle.load(fp)
else:
    print('Preprocessing twits')
    tokenized_twits = [preprocess(message) for message in messages]
    with open(filename, 'wb') as fp:
        pickle.dump(tokenized_twits, fp)

print(f'raw text: {messages[0]}')
print(f'tokenized_twits: {tokenized_twits[0]}')

Load preprocessed twits
raw text: $FITB great buy at 26.00...ill wait
tokenized_twits: ['great', 'buy', 'at', 'ill', 'wait']


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter
import numpy as np

"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
tokenized_words = [word for twit in tokenized_twits for word in twit]

bow = Counter(tokenized_words)

In [8]:
print(bow.most_common(3))

[('the', 398754), ('to', 379487), ('is', 284865)]


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [9]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_cnt = sum(bow.values())
freqs = dict(bow)
for key in freqs:
    freqs[key] = freqs[key]/total_cnt

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5e-6

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word[0] for word in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'are', 'you']


6612

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [10]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word:idx+1 for idx, word in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {idx:word for word, idx in vocab.items()}

# tokenized with the words not in `filtered_words` removed.
filtered = []
filename = 'filteredTwits.txt'
if os.path.exists(filename):
    print('Load preprocessed filteredTwits')
    with open(filename, 'rb') as fp:
        filtered = pickle.load(fp)
else:
    print('Filtering twits')
    for twit in tokenized_twits:
        filtered.append([word for word in twit if word in filtered_words])
    with open(filename, 'wb') as fp:
        pickle.dump(filtered, fp)

Load preprocessed filteredTwits


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [11]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [12]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19401055357594876

Finally let's convert our tokens into integer ids which we can pass to the network.

In [13]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [14]:
print(token_ids[:10])
print(sentiments[:10])

[[1, 2, 3, 4], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 2], [18, 19, 20, 21, 22, 23, 21, 24, 25, 22, 26, 27], [30, 31, 32, 33, 34, 35, 36, 37], [38, 39, 40, 41, 42, 43], [44, 45, 46, 47, 48, 49, 13, 50, 51, 52], [53, 54, 46, 55, 56, 57, 58], [31, 59, 60, 61, 62, 63, 64, 65, 66, 57, 63, 67, 68, 69, 70, 71], [72, 2], [73, 74, 73, 75, 76]]
[4, 4, 3, 3, 0, 3, 4, 0, 4, 0]


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [15]:
train_on_gpu = torch.cuda.is_available()

print("Train on GPU") if train_on_gpu else print("Train on CPU")

Train on GPU


In [16]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement
        
        # Setup embedding layer
        # The input to the module is a list of indices, 
        # and the output is the corresponding word embeddings.
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup LSTM (hidden) layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout = dropout, batch_first = True)
        
        # Fully-connected layer
        self.fc = nn.Linear(lstm_size, output_size)
        self.dropout = nn.Dropout(p=0.3)
        self.log_softmax = nn.LogSoftmax(dim = 1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        if train_on_gpu:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement
        
        # nn_input(batch_size, seq_length)
        embedding_out = self.embedding(nn_input)
        
        # embedding_out(batch_size, seq_length, embed_size)
        # hidden_state(n_layers x batch_size x hidden_dim)
        lstm_out, hidden_state = self.lstm(embedding_out, hidden_state)
        # lstm_out(batch_size, seq_length, lstm_size)
        
        # For each batch, get the last lstm output in the sequence
        lstm_out = lstm_out[:, -1, :]
        
        fc_out = self.dropout(self.fc(lstm_out))
        
        # fc_out(batch_size*seq_length, output_size)
        logps = self.log_softmax(fc_out);
        
        return logps, hidden_state

In [17]:
vocab_size = len(vocab)
embed_size = 10
lstm_size = 6
output_size = 5
lstm_layers= 2

model = TextClassifier(vocab_size, embed_size, lstm_size, output_size, dropout=0.1, lstm_layers = lstm_layers)
print(model)

TextClassifier(
  (embedding): Embedding(6612, 10)
  (lstm): LSTM(10, 6, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (dropout): Dropout(p=0.3)
  (log_softmax): LogSoftmax()
)


### View Model

In [18]:
vocab_size = len(vocab)
embed_size = 10
lstm_size = 6
output_size = 5
lstm_layers= 2

model = TextClassifier(vocab_size, embed_size, lstm_size, output_size, dropout=0.1, lstm_layers = lstm_layers)
model.embedding.weight.data.uniform_(-1, 1)

batch_size = 4
seq_length = 5
input = torch.randint(0, 1000, (batch_size, seq_length), dtype=torch.int64)
if train_on_gpu:
    input = input.cuda()
    model.cuda()
    
hidden = model.init_hidden(batch_size)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-2.1172, -1.1270, -1.9864, -1.5644, -1.5644],
        [-2.0450, -1.1264, -1.9436, -1.5337, -1.6739],
        [-2.0721, -1.5504, -1.9691, -1.0842, -1.6918],
        [-2.2199, -1.2164, -2.0770, -1.1794, -1.8183]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(batch_size, sequence_length)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(25, 40)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [19]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((len(batch_messages), sequence_length), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[batch_num, start_idx:] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [20]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

n_training = np.int(0.8 * len(token_ids))
n_validation = np.int(0.1 * len(token_ids))

train_features = token_ids[:n_training]
valid_features = token_ids[n_training:n_training + n_validation]
train_labels = sentiments[:n_training]
valid_labels = sentiments[n_training:n_training + n_validation]
print(len(train_features), len(valid_labels))

821712 102714


In [21]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
if train_on_gpu:
    text_batch = text_batch.cuda()
    model.cuda()
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(6613, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (dropout): Dropout(p=0.3)
  (log_softmax): LogSoftmax()
)

In [23]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 1
batch_size = 512
sequence_length = 40
learning_rate = 0.001
clip = 5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    hidden = model.init_hidden(batch_size)
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length = sequence_length, shuffle=True):
        
        if text_batch.shape != torch.Size([batch_size, sequence_length]):
            continue
                
        steps += 1
        hidden = tuple([each.data for each in hidden])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        model.zero_grad()
        
        logps, hidden = model.forward(text_batch, hidden)
        
        loss = criterion(logps.squeeze(), labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            
            valid_hidden = model.init_hidden(batch_size)
            valid_loss = []
            # TODO Implement: Print metrics
            for valid_texts, labels in dataloader(
                valid_features, valid_labels, batch_size=batch_size, sequence_length = sequence_length, shuffle=True):
                
                if valid_texts.shape != torch.Size([batch_size, sequence_length]):
                    continue
                
                valid_hidden = tuple([each.data for each in valid_hidden])
                valid_texts, labels = valid_texts.to(device), labels.to(device)
                for each in valid_hidden:
                    each.to(device)
                    
                valid_logps, valid_hidden = model.forward(valid_texts, valid_hidden)
                
                loss = criterion(valid_logps.squeeze(), labels)
                valid_loss.append(loss.item())
            
            print(f'Epoch: ({epoch+1}/{epochs}...)',
                  f'Step: {steps}...',
                  f'Validation loss: {np.mean(valid_loss)}')
            model.train()

Starting epoch 1
Epoch: (1/1...) Step: 100... Validation loss: 1.0021054914593697
Epoch: (1/1...) Step: 200... Validation loss: 0.9045261889696121
Epoch: (1/1...) Step: 300... Validation loss: 0.8630593079328537
Epoch: (1/1...) Step: 400... Validation loss: 0.8263770627975464
Epoch: (1/1...) Step: 500... Validation loss: 0.8017379134893418
Epoch: (1/1...) Step: 600... Validation loss: 0.7890897753834725
Epoch: (1/1...) Step: 700... Validation loss: 0.7752371823787689
Epoch: (1/1...) Step: 800... Validation loss: 0.7584137234091759
Epoch: (1/1...) Step: 900... Validation loss: 0.7579039126634598
Epoch: (1/1...) Step: 1000... Validation loss: 0.7444847962260246
Epoch: (1/1...) Step: 1100... Validation loss: 0.744362855553627
Epoch: (1/1...) Step: 1200... Validation loss: 0.739463067650795
Epoch: (1/1...) Step: 1300... Validation loss: 0.7347753727436066
Epoch: (1/1...) Step: 1400... Validation loss: 0.7320591765642166
Epoch: (1/1...) Step: 1500... Validation loss: 0.7173329085111618
Epoc

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [32]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in filtered_words]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens] 

    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(0)
    
    if train_on_gpu:
        text_input = text_input.cuda()
        model.cuda()
    # Get the NN output
    hidden = model.init_hidden(text_input.size(0))
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [33]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0068,  0.0758,  0.0280,  0.7671,  0.1223]], device='cuda:0')

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
Google has sentiment $3$ with probability $0.7671$

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [34]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [35]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [36]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [37]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1280,  0.1249,  0.1665,  0.1869,  0.3937]], device='cuda:0'),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.